In [1]:
NONE = 0b00000000
EOF = -1
SP_OR_COMM,KW,ID,OP,INT,FP,DEL,STR = 'sp_comm','k','id','op','int','fp','del','str'
YES_SP_OR_COMM, YES_KW, YES_ID, YES_OP, YES_INT, YES_FP, YES_DEL, YES_STR = 0b10000000, 0b01000000, 0b00100000, 0b00010000, 0b00001000, 0b00000100, 0b00000010, 0b00000001

In [2]:
charSet = [
    'a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z',
    'A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z',
    '0','1','2','3','4','5','6','7','8','9','_'
]

TokenNo = 0

spOrCommToken = TokenNo
TokenNo += 1

keywords = ['int', 'float', 'boolean', 'string',
            'while', 'until', 'if else', 'true', 'false']
keywordEnm = list(enumerate(keywords, start=TokenNo))
TokenNo += len(keywords)

idToken = TokenNo
TokenNo += 1

operators = ['+', '-', '*', '/', '%', '?', '>', '<', '!', ':',
            ':=', '==', '>=', '<=', '!=', '&&', '||']
opEnm = list(enumerate(operators, start=TokenNo))
TokenNo += len(operators)

intToken = TokenNo
TokenNo += 1

fpToken = TokenNo
TokenNo += 1

delimiters = ['{', '}', '[', ']', '(', ')', ';', ',']
delimEnm = list(enumerate(delimiters, start=TokenNo))
TokenNo += len(delimiters)

strToken = TokenNo
TokenNo += 1

In [3]:
def tokenNum(list, token):
    for i in list:
        if i[1] == token:
            return i[0]
    return -1

In [4]:
def getNext():
    return ''

In [5]:
def util(str, fp, state, tokenNo):
    return [str, fp, state, tokenNo]

In [6]:
def isKeyword(str):
    fp = 0
    try:
        str[fp]
        for kw in keywords:
            try:
                if str[fp:len(kw)] == kw:
                    fp+=len(kw); str[fp]
                    return (str[:fp], fp, YES_KW, tokenNum(keywordEnm, str[:fp]))
            except(IndexError):
                    return (str[:fp], EOF, YES_KW, tokenNum(keywordEnm, str[:fp]))
                
        return ('', 0, 0b00000000, -1)

    except(IndexError):
        return ('', EOF, 0b00000000, -1)
print(isKeyword('int'))

('int', -1, 64, 1)


In [7]:
def isDelimiter(str):
    fp = 0
    state = DEL
    try:
        if str[fp] in delimiters:
            fp+=1
            state = YES_DEL
        
        if state == YES_DEL:
            try:
                str[fp]
                return (str[0], fp, state, tokenNum(delimEnm, str[0]))
            except(IndexError):
                return (str[0], EOF, state, tokenNum(delimEnm, str[0]))
        
        else:
            return ('',0,NONE, -1)
                    
    except(IndexError):
        return ('', EOF, NONE, -1)
print(isDelimiter('[['))

('[', 1, 2, 32)


In [8]:
def isOp(str):
    state = OP
    fp = 0
    try:
        while True:
            if state == OP:
                if str[fp] in operators[:6]:
                    state = YES_OP
                elif str[fp] == ':':
                    state = ':'
                elif str[fp] == '>':
                    state = '>'
                elif str[fp] == '<':
                    state = '<'
                elif str[fp] == '!':
                    state = '!'
                elif str[fp] == '=':
                    state = '=='
                elif str[fp] == '&':
                    state = '&&'
                elif str[fp] == '|':
                    state = '||'
                else:
                    return ('', 0, NONE, -1)
                fp+=1
                
            elif state == YES_OP:
                try:
                    str[fp]
                    return (str[:fp], fp, YES_OP, tokenNum(opEnm, str[:fp]))
                except(IndexError):
                    return (str[:fp], EOF, YES_OP, tokenNum(opEnm, str[:fp]))
                    
            elif state==':' or state=='>' or state=='<' or state=='!':
                try:
                    if str[fp] == '=':
                        fp+=1
                        state = YES_OP
                    else:
                        state = YES_OP
                except(IndexError):
                    state = YES_OP
            
            elif state == '==':
                try:
                    if str[fp] == '=':
                        fp+=1
                        state = YES_OP
                    else:
                        return ('', 0, NONE, -1)
                except(IndexError):
                        return ('', 0, NONE, -1)
                    
                
            elif state == '&&':
                try:
                    if str[fp] == '&':
                        fp+=1
                        state = YES_OP
                    else:
                        return ('', 0, NONE, -1)
                except(IndexError):
                        return ('', 0, NONE, -1)
                    
                
            elif state == '||':
                try:
                    if str[fp] == '|':
                        fp+=1
                        state = YES_OP
                    else:
                        return ('', 0, NONE, -1)
                except(IndexError):
                        return ('', 0, NONE, -1)
                    
                
    except(IndexError):
        return ('', EOF, NONE, -1)
print(isOp('=='))

('==', -1, 16, 22)


In [9]:
def isId(str):
    state = ID
    fp = 0
    try:
        while True:
            if state == ID:
                if str[fp] in charSet[:26]:
                    state = YES_ID
                    fp+=1
                else:
                    return ('', fp, NONE, -1)
            
            elif state == YES_ID:
                try:
                    if str[fp] in charSet:
                        state = YES_ID
                        fp+=1
                    else:
                        return (str[:fp], fp, state, idToken)
                except(IndexError):
                    return (str[:fp], EOF, state, idToken)
    
    except(IndexError):
        return ('', EOF, NONE, -1)    

print(isId(';'))

('', 0, 0, -1)


In [10]:
def isInt(str):
    state = INT
    fp = 0
    try:
        while True:
            if state == INT:
                if str[fp]=='+' or str[fp]=='-':
                    state = '1'
                    fp+=1
                elif str[fp] in charSet[53:62]:
                    state = '2'
                    fp+=1
                elif str[fp] == '0':
                    state = '3'
                    fp+=1
                else:
                    return ('', fp, NONE, -1)
            
            elif state == '1':
                if str[fp] in charSet[53:62]:
                    state = '2'
                    fp+=1
                else:
                    return ('', fp, NONE, -1)
            
            elif state == '2':
                try:
                    if str[fp] in charSet[52:62]:
                        state = '2'
                        fp+=1
                    else:
                        state = YES_INT
                except(IndexError):
                    state = YES_INT
        
            elif state == '3':
                try:
                    if str[fp] == '0':
                        state = '3'
                        fp+=1
                    elif str[fp] in charSet[53:62]:
                        state = '4'
                        fp+=1
                    else:
                        state = YES_INT
                except(IndexError):
                    state = YES_INT
                
            elif state == '4':
                return ('', fp, NONE, -1)

            elif state == YES_INT:
                try:
                    str[fp]
                    return (str[:fp], fp, state, intToken)
                except(IndexError):
                    return (str[:fp], EOF, state, intToken)
            
    except(IndexError):
        return ('', EOF, NONE, -1)
print(isInt('+123f'))

('+123', 4, 8, 28)


In [11]:
def isFP(str):
    state = FP
    fp = 0
    try:
        while True:
            if state == FP:
                if str[fp] == '+' or str[fp] == '-':
                    state = '1'
                    fp += 1
                elif str[fp] in charSet[53:62]:
                    state = '2'
                    fp += 1
                elif str[fp] == '0':
                    state = '6'
                    fp += 1
                else:
                    return ('', fp, NONE, -1)

            elif state == '1':
                if str[fp] in charSet[53:62]:
                    state = '2'
                    fp += 1
                elif str[fp] == '0':
                    state = '8'
                    fp+=1
                else:
                    return ('', fp, NONE, -1)

            elif state == '2':
                if str[fp] in charSet[52:62]:
                    state = '2'
                    fp += 1
                elif str[fp] == '.':
                    state = '3'
                    fp+=1
                else:
                    return ('', fp, NONE, -1)

            elif state == '3':
                if str[fp] == '0':
                    state = '4'
                    fp += 1
                elif str[fp] in charSet[53:62]:
                    state = YES_FP
                    fp+=1
                else:
                    return ('', fp, NONE, -1)
            
            elif state == '4':
                if str[fp] in charSet[53:62]:
                    state = YES_FP
                    fp+=1
                elif str[fp] == '0':
                    state = '4'
                    fp+=1
                else:
                    return ('', fp, NONE, -1)

            elif state == YES_FP:
                try:
                    if str[fp] in charSet[52:62]:
                        state = YES_FP
                        fp+=1
                    else:
                        return (str[:fp], fp, state, fpToken)
                except(IndexError):
                    return (str[:fp], EOF, state, fpToken)
        
            elif state == '6':
                if str[fp] == '.':
                    state = '7'
                    fp+=1
                else:
                    return ('', fp, NONE, -1)
                
            elif state == '7':
                if str[fp] in charSet[52:62]:
                    state = YES_FP
                    fp+=1
                else:
                    return ('', fp, NONE, -1)
                
            elif state == '8':
                if str[fp] == '.':
                    state = '3'
                    fp+=1
                else:
                    return ('', fp, NONE, -1)
    except(IndexError):
        return ('', EOF, NONE, -1)

print(isFP('+0'))

('', -1, 0, -1)


In [12]:
def isStr(str):
    state = STR
    fp = 0
    try:
        while True:
            if state == STR:
                if str[fp] == '"':
                    state = '1'
                    fp+=1
                else:
                    return ('', fp, NONE, -1)
            
            elif state == '1':
                if str[fp] == '\\':
                    state = '3'
                    fp+=1
                elif str[fp] == '"':
                    state = YES_STR
                    fp+=1
                elif str[fp] not in ['\n', '\r', '\\', '"']:
                    state = '1'
                    fp+=1
                else:
                    return ('', fp, NONE, -1)
                    
            elif state == YES_STR:
                try:
                    str[fp]
                    return (str[:fp], fp, state, strToken)
                except(IndexError):
                    return (str[:fp], EOF, state, strToken)
                    
            
            elif state == '3':
                if str[fp] in ['\\','n','r','"','t']:
                    state = '1'
                    fp+=1
                else:
                    return ('', fp, NONE, -1)

    except(IndexError)            :
        return ('', EOF, NONE, -1)

with open('test2.txt', 'r') as f:
    x = f.read()
    print(x)
    print(isStr(x))


int main(){
    int x := 5;
    string st := "asf\n";

    #dsfnjfn

    float := +2.3;
}
('', 0, 0, -1)


In [13]:
def isSpaceOrComm(str):
    state = SP_OR_COMM
    fp = 0
    lines = 0
    try:
        while True:
            if state == SP_OR_COMM:
                if str[fp] in [' ', '\n', '\t', '\r']:
                    if str[fp] == '\n':
                        lines+=1
                    state = 'sp'
                    fp+=1
                elif str[fp] == '#':
                    state = 'comm'
                    fp+=1
                else:
                    return ('', fp, NONE, -1)
            
            elif state == 'sp':
                try:
                    if str[fp] in [' ', '\n', '\t', '\r']:
                        if str[fp] == '\n':
                            lines+=1
                        state = 'sp'
                        fp+=1
                    else:
                        state = YES_SP_OR_COMM
                except(IndexError):
                    state = YES_SP_OR_COMM
            
            elif state == 'comm':
                try:
                    if str[fp] != '\n':
                        state = 'comm'
                        fp+=1
                    else:
                        fp+=1
                        lines+=1
                        state = YES_SP_OR_COMM
                except(IndexError):
                    state = YES_SP_OR_COMM
            
            elif state == YES_SP_OR_COMM:
                try:
                    str[fp]
                    return ('', fp, state, lines)
                except(IndexError):
                    return ('', EOF, YES_SP_OR_COMM, lines)
                
    except(IndexError):
        return ('', EOF, NONE, -1)
            
print(isSpaceOrComm('#sklj\n'))

('', -1, 128, 1)


In [14]:
def panic_mode(str):
    fp = 0
    while True:
        try:
            if str[fp] == '\n':
                return fp+1
            else:
                fp+=1
        except(IndexError):
            exit()

In [15]:
def binUtil(str):
    str = bin(str)[2:]
    if len(str) < 8:
        for i in range(8-len(str)):
            str = '0' + str
    return str

In [20]:
IND_SP_COMM, IND_KW, IND_ID, IND_OP, IND_INT, IND_FP, IND_DEL, IND_STR = 0, 1, 2, 3, 4, 5, 6, 7

TokenNames = {
    IND_SP_COMM: ['Space / Comment', isSpaceOrComm],
    IND_KW: ['Keyword', isKeyword],
    IND_ID: ['Identifier', isId],
    IND_OP: ['Operator', isOp],
    IND_INT: ['Integer Literal', isInt],
    IND_FP: ['Floating Point Literal', isFP],
    IND_DEL: ['Delimiter', isDelimiter],
    IND_STR: ['String Literal', isStr]
}
def dfa():
    lp, eof = 0, -1
    with open('test2.txt', 'r') as f:
        codex = f.read()
        lineNo = 1
        while eof == -1:
            tokens, state, final = [], 0b00000000, None
            
            for i in range(8):
                tokens.append(TokenNames[i][1](codex))
                state |= tokens[i][2]
                
            state = binUtil(state)
            
            if state[IND_KW]=='1' and state[IND_ID]=='1':
                if tokens[IND_KW][1] >= tokens[IND_ID][1]:
                    final = tokens[IND_KW]
                else:
                    final = tokens[IND_ID]
            elif state[IND_INT] == '1' and state[IND_FP] == '1' and state[IND_OP] == '1':
                final = tokens[IND_OP]
            elif state[IND_INT] == '0' and state[IND_FP] == '1' and state[IND_OP] == '1':
                final = tokens[IND_OP]
            elif state[IND_INT] == '1' and state[IND_FP] == '0' and state[IND_OP] == '1':
                final = tokens[IND_OP]
            elif state[IND_INT] == '1' and state[IND_FP] == '1' and state[IND_OP] == '0':
                final = tokens[IND_FP]
            else:
                for index in range(8):
                    if state[index] == '1':
                        if index == IND_SP_COMM:
                            lineNo += tokens[index][3]
                        final = tokens[index]
                        break
            
            if final==None:
                lp = panic_mode(codex)
                codex = codex[lp:]
                lineNo += 1
            else:
                if final[1]==EOF:
                    eof = 1
                else:
                    lp = final[1]
                codex = codex[lp:]
                if final[2] != 0b10000000:
                    print(lineNo, final[0], final[2])

dfa()

1 int 64
1 main 32
1 ( 2
1 ) 2
1 { 2
2 int 64
2 x 32
2 := 16
2 5 8
2 ; 2
3 string 64
3 st 32
3 := 16
3 "asf\n" 1
3 ; 2
7 float 64
9 } 2
